In [1]:
import os
import findspark
findspark.init()

from collections import namedtuple

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc


os.environ['SPARK_HOME'] = '/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [2]:
sc = SparkContext()
ssc = StreamingContext(sc, 10)
sql_context = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/14 11:48:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/.venv/lib/python3.10/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/.venv/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

In [4]:
lines = socket_stream.window(20)

In [5]:
fields = ('tag', 'count')
tweet = namedtuple('Tweet', fields)

In [6]:
(
    lines.flatMap(lambda line: line.split(" "))
    .filter(lambda word: word.lower().startswith("#"))
    .map(lambda hashtag: (hashtag.lower(), 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda record: tweet(record[0], record[1]))
    .foreachRDD(lambda rdd: rdd.toDF().sort(desc('count')).limit(10).registerTempTable('tweets'))
)

# ploting

In [7]:
import time
from IPython import display

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [8]:
%matplotlib inline

In [9]:
ssc.start()


23/11/14 11:49:31 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark/python/lib/pyspark.zip/pyspark/worker.py", line 810, in main
    eval_type = read_int(infile)
  File "/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:767)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:749)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.

In [ ]:
count = 0
while count < 10:
    time.sleep(3)
    top_10 = sql_context.sql("SELECT tag, count from tweets")
    top_10_pd_df = top_10.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize= (10, 8))
    sns.barplot(x='count', y='tag', data=top_10_pd_df)
    plt.show()
    count = count + 1

    

In [ ]:
ssc.stop()
sc.stop()